## Daily racing stats scraping notebook
***

In [86]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from currency_converter import CurrencyConverter

import json 
import hashlib
from typing import List

import time
import random
import decimal

In [75]:

race_cols = [
    "date",
    "track", 
    "race_name", 
    "race_age_group", 
    "race_class", 
    "distance", 
    "going", 
    "runners", 
    "track_type", 
    "race_type",
    "off_time", 
    "winning_time",
    "prize_money_json", 
    "purse"
]

horse_cols = [
    "race_id",
    "horse_id",  
    "horse_name",
    "horse_age",
    "horse_sex",  
    "dam", 
    "sire", 
    "owner", 
    "trainer", 
    "jockey", 
    "weight", 
    "sp",
    "position", 
    "beaten_by", 
    "winnings", 
    "stall_number", 
    "horse_running_description",
    "estimated_running_time"
]

In [76]:
race_df = pd.DataFrame(columns=race_cols)
horse_df = pd.DataFrame(columns=horse_cols)

In [87]:
class Horse:
    def __init__(self, browser):
        self.browser = browser 
        
    def get_horse_id(self):
        hash_object = hashlib.md5(self.horse_name.encode())
        return hash_object.hexdigest()
        
    def get_runner_info(self, runner_info):
        if len(runner_info) == 9:
            [position, horse_number, horse_name, age_weight, sp, trainer_jockey, race_comments, text, text2] = runner_info 
            beaten_by = 'N/A'
        elif len(runner_info) == 10:
            [position, beaten_by, horse_number, horse_name, age_weight, sp, trainer_jockey, race_comments, text, text2] = runner_info
            
        if age_weight[0] in [1,2]:
            weight = age_weight[1:]
            age = age_weight[0]
        else:
            age = age_weight[0:2]
            weight = age_weight[2:]
            
        tj_split = trainer_jockey.split('J: ')
        self.trainer = tj_split[0].replace('T: ', '')
        self.jockey = tj_split[1]
        self.position= position
        self.horse_name = horse_name
        self.horse_id = self.get_horse_id() 
        self.horse_age = age 
        self.horse_weight = weight
        self.beaten_by = beaten_by 
        self.sp = sp 
        self.race_comments = race_comments
        
        self.race_id = "TODO"
        self.winnings = "TODO"
        self.stall_number = "TODO"
        self.estimated_running_time = "TODO"
        
    def get_breeding_info(self):
        # Links to click on the horse 
        horseLinks = self.browser.find_elements(By.XPATH, "//a")
        selectedHorse = [i for i in horseLinks if i.text == self.horse_name]
        if len(selectedHorse) > 0:
            browser.execute_script("arguments[0].click();", selectedHorse[0])

        time.sleep(float(decimal.Decimal(random.randrange(100, 500))/100))
        # get breeding info for the horse
        breeding_info = browser.find_elements(By.CSS_SELECTOR, "table[class^='Header__DataTable']")[0].text.split('\n')
        [age2, trainer2, horse_sex, sire, dam, owner] = breeding_info 

        self.horse_sex = horse_sex.split(' ')[1]
        self.sire = " ".join(sire.split(' ')[1:])
        self.dam = " ".join(dam.split(' ')[1:])
        self.owner = " ".join(owner.split(' ')[1:])
        self.browser.back()
        time.sleep(float(decimal.Decimal(random.randrange(100, 500))/100))
        
    def add_to_df(self, df):
        """ 
        appends row to pandas dataframe
        TODO: this will change to append row to postgres table using sqlalchemy
        """
        row_dict = {
            "race_id": self.race_id,
            "horse_id": self.horse_id,  
            "horse_name": self.horse_name,
            "horse_age": self.horse_age,
            "horse_sex": self.horse_sex,  
            "dam": self.dam, 
            "sire": self.sire, 
            "owner": self.owner, 
            "trainer": self.trainer, 
            "jockey": self.jockey, 
            "weight": self.horse_weight, 
            "sp": self.sp,
            "position": self.position, 
            "beaten_by": self.beaten_by, 
            "winnings": self.winnings, 
            "stall_number": self.stall_number, 
            "race_comments": self.race_comments,
            "estimated_running_time": self.estimated_running_time
        }
        df = df.append(row_dict, ignore_index=True)
        return df
        
class Race:
    def __init__(self, browser):
        self.browser=browser
        
    def get_race_info(self):
        self.browser.current_url.split('/')
        self.date = self.browser.current_url.split('/')[5]
        self.track = self.browser.current_url.split('/')[6]

        race_info = self.browser.find_elements(By.CSS_SELECTOR, "div[class^='RacePage__SummaryWrapper']")[0].text.split('\n')
        
        self.race_name = race_info[0]
        race_sub_info = self.get_variable_race_sub_info(race_info[1].split('  |   '))
        self.race_age_group = race_sub_info[0]
        self.race_class = race_sub_info[1]
        self.distance = self.get_distance_in_yards(race_sub_info[2])
        self.going = race_sub_info[3]
        self.runners = race_sub_info[4].split(' ')[0]
        self.track_type = race_sub_info[5]
        timing = race_info[3].split('  |   ') 
        self.off_time = timing[0].split(': ')[1]
        self.winning_time = self.get_time_in_seconds(timing[1].split(': ')[1])
        self.race_type='Need to dev'
        prize_money = self.get_prize_money_dict()
        self.prize_money_json = json.dumps(prize_money)
        self.purse = sum(prize_money.values())
        
    def get_variable_race_sub_info(self, race_sub_info: List[str]) -> List[str]:
        """ 
        Deals with issue where Irish courses don't use race class so need to set that as N/A
        """
        if len(race_sub_info) == 5:
            [race_age_group, distance, going, num_runners, track] = race_sub_info
            class_of_race = 'N/A'  
        else:
            [race_age_group, class_of_race, distance, going, num_runners, track] = race_sub_info
        
        return [race_age_group, class_of_race, distance, going, num_runners, track] 
        
        
    def get_time_in_seconds(self, winning_time: str) -> float:
        """
        Input looks like this '5m 6.71s' -> convert to total seconds
        """
        if len(winning_time.split(' ')) == 1:
            seconds = float(winning_time.replace('s',''))
            self.winning_time = seconds
            return seconds
        else:
            [min, sec] = winning_time.split(' ')
            min_to_seconds = int(min.replace('m', '')) * 60
            seconds = float(sec.replace('s',''))
            total_time =  min_to_seconds + seconds
            self.winning_time = total_time
            return total_time
        
    def get_distance_in_yards(self, distance: str) -> int:
        """ 
        Converts the race distance to yards for ease of analysis
        """
        segments = distance.split(' ')
        race_yards = 0
        for item in segments:
            if 'm' in item:
                miles = item.replace('m','')
                miles_to_yards = int(miles) * 1760
                race_yards += miles_to_yards
            elif 'f' in item: 
                furlongs = item.replace('f', '')
                furlongs_to_yards = int(furlongs) * 220 
                race_yards += furlongs_to_yards 
            elif 'y' in item:
                yards = item.replace('y', '')
                race_yards += int(yards)
        return race_yards
    
    def get_winnings_in_gbp(self, winnings_string: str) -> float:
        """ 
        Where prize money is in euros, convert to gbp using the currency converter API 
        (uses European Central Bank rates)
        """
        c = CurrencyConverter()
        winning_val = winnings_string[1:].replace(',','')
        if winnings_string[0] == '€':
            winnings = float(winning_val)
            return c.convert(winnings, 'EUR', 'GBP')
        else: 
            return float(winning_val)
    
    def get_prize_money_dict(self) -> dict:
        """ 
        Gets the prize money for the race and returns dict of {'position': 'winnings'}
        """
        prize_money_elements = self.browser.find_elements(By.CSS_SELECTOR, "div[class^='PrizeMoney__PrizeSummary-sc-199orl7-3']")[0].text.split('\n')
        prize_money = {} 
        for i, item in enumerate(prize_money_elements):
            if (i % 2 == 0) | (i == 0):
                key = prize_money_elements[i][:-3] #removes text just leaves the position number as a string
                val = prize_money_elements[i+1] 
                prize_money[key] = self.get_winnings_in_gbp(winnings_string = val) 
                
        return prize_money
    
    def add_to_df(self, df):
        """ 
        appends row to pandas dataframe
        TODO: this will change to append row to postgres table using sqlalchemy
        """
        row_dict = {
            "date": self.date,
            "track": self.track, 
            "race_name": self.race_name, 
            "race_age_group": self.race_age_group,
            "race_class": self.race_class,
            "distance": self.distance,
            "going": self.going,
            "runners": self.runners,
            "track_type": self.track_type,
            "race_type": self.race_type,
            "off_time": self.off_time,
            "winning_time": self.winning_time, 
            "prize_money_json": self.prize_money_json, 
            "purse": self.purse
        }
        df = df.append(row_dict, ignore_index=True)
        return df
        

In [82]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.sportinglife.com/racing/results/yesterday')
time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
cookies_button = browser.find_elements(By.CSS_SELECTOR, "button[class^='BaseButton__BaseButtonStyled-e225m1-0']")
if len(cookies_button) > 0:
    cookies_button[0].click() 
num_races = len(browser.find_elements(By.CSS_SELECTOR, "span[class^='Race__RaceTime-sc-16yubq3-1']"))
race_selectors = ["span[class^='Race__RaceTime-sc-16yubq3-1']", 
                  "div[class^='FutureRace__RaceName-sc-1yen8s9-0']", 
                  "div[class^='FutureRace__RaceDetailsContainer-sc-1yen8s9-1']"] # this gives impression of clicking on different parts of the race button

for i in range(0, num_races):
    # First get info for the race itself
    timeout = 10
    idx = random.randint(0,2)
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, race_selectors[idx])))
    browser.refresh()
    time.sleep(float(decimal.Decimal(random.randrange(100, 200))/100))
    races = browser.find_elements(By.CSS_SELECTOR, race_selectors[idx])
    browser.execute_script("arguments[0].click();", races[i])
    time.sleep(float(decimal.Decimal(random.randrange(500, 1000))/100))
    new_race = Race(browser)
    new_race.get_race_info()
    race_df = new_race.add_to_df(race_df)
    
    # Now get info for each runner in the race
    num_runners = len(browser.find_elements(By.CSS_SELECTOR, "div[class^='ResultRunner__StyledResultRunner']") )
    for runs_idx in range(0, num_runners):
        browser.refresh()
        time.sleep(float(decimal.Decimal(random.randrange(100, 200))/100))
        runner = browser.find_elements(By.CSS_SELECTOR, "div[class^='ResultRunner__StyledResultRunner']")[runs_idx]
        runner_info = runner.text.split('\n')
        horse = Horse(browser) 
        horse.get_runner_info(runner_info) 
        horse.get_breeding_info()
        horse_df = horse.add_to_df(horse_df)
    
    browser.back()
    time.sleep(float(decimal.Decimal(random.randrange(500, 1200))/100))
    print(f'{i+1}/{num_races}')


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [/Users/jackpickard/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/jf/wwr3jnbx3zvfghjk_j_tbxnm0000gp/T/ipykernel_72909/3561360229.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


1/30
2/30
3/30
4/30
5/30
6/30
7/30
8/30
9/30
10/30
11/30
12/30
13/30
14/30


NameError: name 'c' is not defined

In [84]:
horse_df

,race_id,horse_id,horse_name,horse_age,horse_sex,dam,sire,owner,trainer,jockey,weight,sp,position,beaten_by,winnings,stall_number,horse_running_description,estimated_running_time,race_comments
0,TODO,9665d9754e9d16275159dc3cbc360cc0,Balco Coastal,51,Gelding,Fliugika,Coastal Path,Mr Mark Blandford,N J Henderson,Nico de Boinville,0-12,8/11f,1st,N/A,TODO,TODO,NaN,TODO,"Chased leaders, headway into 2nd well over 2 o..."
1,TODO,8e44f6c33c92c074c198a16a40d5d99a,Extraordinary Man,51,Gelding,Argovie,No Risk At All,Mr Simon Hunt,D G Bridgwater,T Scudamore,0-12,150/1,2nd,6,TODO,TODO,NaN,TODO,"Tracked leader, lost place and chased leaders ..."
2,TODO,e3daa94d0117c2426d6cad0fac16f752,Peejaybee,51,Gelding,Playa Du Charmil,Ballingarry,Martin Gowing And Paul Booker,R Spencer,James Bowen,1-5,15/2,3rd,2 ¼,TODO,TODO,NaN,TODO,"Chased leaders, pushed along well over 2 out, ..."
3,TODO,541ca838b671bb98c5a35b6fa5c6d99d,Black Poppy,51,Gelding,Poppy Come Running,Kayf Tara,West Coast Haulage Limited,Kerry Lee,Richard Patrick,0-12,5/1,4th,3 ¼,TODO,TODO,NaN,TODO,"Chased leaders on outer early, headway and tra..."
4,TODO,e33b3f3a7ca4271c221eb7f36ba77b55,Gentleman At Arms,41,Gelding,Sworn Sold,Reliable Man,D & B Partnership,S Edmunds,C Gethings,0-12,25/1,5th,6 ½,TODO,TODO,NaN,TODO,"Mid-division, closer after 3rd, pushed along u..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,TODO,8311b7849be06d206b0cc552f968bb63,Duty Calls,81,Gelding,Inniskeen,Arcadio,Dont Mind If We Do,Miss S E Forster,Lewis Stones,0-9,8/1,2nd,1,TODO,TODO,NaN,TODO,"Led, not fluent 3rd, joined 4 out, led again b..."
131,TODO,47bdf8526e5e675eff0e6bee21c1bc01,Lock Down Luke,51,Gelding,La Grande Villez,Lucarno,Mr P Stephen & Mr Mark Fleming,Mrs J Stephen,Tom Midgley,1-1,11/4,3rd,¾,TODO,TODO,NaN,TODO,"In rear, ridden in 4th after 3 out, pressed le..."
132,TODO,4964347a343ebad70f5da93eefb470a8,Malpas,61,Gelding,Skipping Along,Milan,Mr N Hartley,D McCain Jnr,Peter Kavanagh (6),1-12,9/4j,4th,1 ¾,TODO,TODO,NaN,TODO,"Handy early, dropped to rear in touch before 5..."
133,TODO,fce9c8e21eb711141083b045fd3ee69b,Blooriedotcom,61,Gelding,Peaceful Kingdom,Holy Roman Emperor,Mutual Friends,Miss Lucinda V Russell,P W Wadge (10),1-7,11/1,5th,15,TODO,TODO,NaN,TODO,"Prominent, joined leader 4 out, headed before ..."


In [79]:
horse_df 

,race_id,horse_id,horse_name,horse_age,horse_sex,dam,sire,owner,trainer,jockey,weight,sp,position,beaten_by,winnings,stall_number,horse_running_description,estimated_running_time
